# Теория игр

$%![three-in-a-row](http://scientopia.org/img-archive/goodmath/img_270.png)$

Начнём с самого баянного примера математической игры, который можно вспомнить:

> На столе лежит кучка из $n$ спичек. Два игрока по очереди берут спички из кучки, разрешается взять одну, две или три спички. Тот, кто берет последнюю спичку, выигрывает. Требуется определить, кто выигрывает при оптимальной игре обоих игроков.

Все задачи такого типа решаются одним и тем же способом: посмотреть на решения для маленьких чисел и применить динамическое программирование. В данном случае состоянием динамики будет $f_k$ — выиграет ли первый игрок, если спичек сейчас $n$, и ход принадлежит ему. Пересчитывать её нужно так:

$$ f_k = \lnot f_{k-1} \lor \lnot f_{k-2} \lor \lnot f_{k-3} = \lnot (f_{k-1} \land f_{k-2} \land f_{k-3}) $$

то есть должен найтись переход в какое-то состояние динамики, которое является проигрышным (или, что эквивалентно, не должно быть пере динамики должны вести в проигрышные состояния).

**Примечание.** Конечно, здесь не нужна была никакая динамика, и ответ зависит только от того, делится ли $n$ на 4, но мы всё равно рассмотрим этот пример в педагогических целях.

## Игры на графах

Любую игру можно описать в виде графа (возможно, бесконечного) *состояний игры*, между которыми есть переходы. Игроки поочередно выбирают, по какому переходу пройти. Некоторые состояния в этом графе помечены как *терминальные*

Состояние называется *выигрышным*, если игрок, начинающий в нём, побеждает, и *проигрышным* в противном случае. В графе могут быть циклы, и иногда обоим игрокам выгодно по ним ходить. Например, в шахматах бывают ситуации, когда выгодно повторение ходов (сама игра при этом всё равно не может длиться бесконечно, потому что троекратное повторение позиции приводит к ничье — подобные правила есть в большинстве стратегических игр). Вершины, принадлежащие к таким «выгодным» циклам, назовём ничейными.

Подобные игры — где чтобы сделать одному игроку лучше, нужно сделать другому на столько же хуже — называют *играми с нулевой суммой* (почему такое название и что такое игры с ненулевой суммой — ниже прочитаете). Они почти всегда решаются определенным видом динамики на графе, который называется *ретроанализом*.

## Ретроанализ

Обобщим пример со спичками и сформулируем несколько условий выгрышности:

* Все переходы из состояния ведут в проигрышные вершины.

Критерий проигрышности:

* Все переходы ведут в выгрышные состояния

Мы уже имеем играть в ациклические игры — раз циклов нет, значит точно нет и ничейных вершин, и можно просто пойти с конца: пройтись bfs-ом по графу, 

In [ ]:
vector<int> g[maxn], t[maxn]; // списки смежности прямого и обратного графа
int win[maxn]; // выгрышность вершины;
// по умолчанию все кроме терминальных считаются проигрышными, пока не найдётся переход в другую проигрышную

queue<int> q = {/* нужно заранее добавить сюда все терминальные вершины*/};

while (!q.empty()) {
    int v = q.front();
    q.pop();
    for (int u : t[v])
        // если вершина проигрышная, что все её родители выигрышные
        if (!win[v])
            win[u] = true;
        // если же вершина выигрышная, то никто в неё всё равно не захочет ходить
}

Как обычно это бывает с динамиками, иногда проще написать рекурсивный подсчёт динамики, а не итеративный. Именно такую реализацию мы будем считать каноничной, потому что она в дальнейшем позволит делать отсечения:

Но что делать, когда есть циклы?

Критерий ничейности

Как по произвольному (конечному) графу определить, какие из его вершин являются выигрышными, проигрышными или ничейными?

## Минимаксные игры

Иногда у нас более богатая градация терминальных состояний, чем проигрышные и выигрышные.



[Игра финансистов](http://codeforces.com/contest/729/problem/F)

## Ретроанализ для больших графов

Рассмотрим какую-нибудь сложную игру. Например, шахматы или крестики-нолики. Мы уже обсудили, что так как это конечные ациклические игры, в них однозначно определен победитель при оптимальной игре. Достаточно просто запустить рекурсивную функцию с функцией `best_turn(board, player)`, которая возращает пару `(turn, game_result)` - лучший ход игрока `player` на доске`board`, причем в оптимальном случае игра заканчивается с результатом `result`. Причем с помощью ленивой динамики можно сохранять результат, чтобы не перебирать одну доску несколько раз. Этот алгоритм, вкратце описанный ранее, еще называют **ретроанализ**.

Понятна проблема такого подхода: **алгоритм работает очень долго**, особенно в случае с играми типа шахмат.

Есть несколько оптимизаций ретроанализа:

**1) Перебирать только на $K$ ходов вперед. **

Для этого надо идти по состояниям графа BFS-ом,  а не DFS-ом, и останавливаться в момент, когда пройдено $K$ ходов. При этом возникает только одна проблемы: как понять результат игры на рассматриваемой доске через $K$ ходов, ведь игра еще не закончена. Для этого надо придумать приближенную функцию которая **численно оценивает, насколько первый игрок выигрывает**. 

Для шахмат можно просто присвоить каждой фигуре сколько-то баллов и считать для каждого игрока эту сумму, а за функцию взять разность сумм у первого и второго игрока. Тогда если у первого игрока есть стратегия, с помощью которой он за $K$ ходов гарантированно ест ферзя, не теряя много своих фигур, ретроанализ это обнаружит.

Ясно, что это дает существенное ускорение по времени: можно подобрать $K$ так, чтобы время было достаточно маленьким. А если есть какой-то определенный лимит по времени, можно прекращать перебирать, когда закончится этот лимит. Но ход, который предлагает ретроанализ с такой оптимизацией, конечно, перестает быть оптимальным. Впрочем, оптимальную стратегию для шахмат пока найти не смогли (а для шашек, кстати, нашли).

**2) Альба-бета-отсечения**

<img src="https://drive.google.com/uc?export=view&id=1TKGO9CA3jgSwXCNBM-ZIWYfablGKcyfW">

На картинке изображена игра с нулевой суммой: в ней сумма выигрыша первого и второго игрока равна нулю, поэтому на картинке в вершине написан только выигрыш того игрока, который ходит из нее первым.

Идею оптимизации рассмотрим на частном примере: мы перебираем куда идти из состояния $X$ (например, корень). Мы уже зашли в первого и второго сына, увидели, что если пойти в первого, то выигрыш второго игрока составит -3, а если во второго, то выигрыш составит -6. Наша задача - минимизровать его выигрыш, так как это максимизирует наш выигрыш. Пока что лучшим ходом из корня дерева является второй сын, и выигрыш будет равен 6.

Давайте зайдем в третьего сына, назовем его $Y$, вдруг там второй сможет набрать больше, чем -6. Зайдем в первого сына $Y$, обработаем полностью и заметим, что при таком ходе второй при оптимальной игре получит -5. Утверждается, что тогда ход во второго сына $Y$ можно вообще не проверять. Почему? Потому что уже понятно, что если пойти из корня $X$ в вершину $Y$, то выигрыш второго будет хотя бы -5 (а возможно даже больше), а значит выигрыш первого будет не более, чем 5. Но ход во второго сына $X$ уже дает 6, что больше, и рассматривать вершину $Y$ далее бессмысленно, мы в нее уже точно не пойдем.

Отсечение таких лишних веток и называется альфа-бета-отсечением. Понятно, что оно может значительно ускорить перебор, так как мы выкидываем целые ветки перебора, и при этом эта оптимизация все еще находит оптимальный ход, в отличие от прошлой оптимизации.

## Простые игры

Есть два игрока и N спичек. Они по очереди берут 1, 2, 3 или 4 спички. Если осталось 0 спичек и игрок походить не может - он проиграл. Кто победит при оптимальной игре?

## Выигрышные и проигрышные состояния

Обычно игра состоит из каких-то **состояний игры**, между которыми есть переходы. В нашем случае состоянием назовем число спичек. Каждый из двух игроков выбирает, куда пойти из текущего состояния, причем из некоторых из них ходить некуда, и тот, кто должен из них ходить - проиграл. 

Давайте назовем **проигрышным состоянием** такое состояние, что игрок, начинающий в нем (первый), проиграет, как бы он ни ходил при оптимальной игре второго. То есть у второго существует такая стратегия, при которой он победит первого в любом случае, если он начнет в проигрышном состоянии.

А **выигрышным состоянием** назовем состояние, в котором игрок, начинающий в нем, имеет стратегию, при которой он в любом случае выиграет.

По сути, ранее описанные win[n] = 1 - это и есть выигрышные состояния, а win[n] = 2 - это проигрышные состояния.

Заметим, что 0 спичек - это точно проигрышное состояние, это и есть база динамики. Также заметим, что
* Если из состояния есть хотя бы один переход в проигрышное состояние, то состояние **выигрышное**
* Если из состояния все переходы ведут в выигрышные состояния, то состояние **проигрышное**

Это довольно важное, но простое наблюдение - есть есть ход в проигрышное состояние, то туда надо срочно идти, ведь второй игрок после этого будет ходить из прогрышного состояния, а значит он проиграет. И наоборот, если все пути ведуь в выигрышное состояние, то как бы мы ни пошли, второй сможет выиграть после этого, а значит такое состояние точно проигрышное.

Вот мы и придумали переход для нашей динамики!

In [0]:
win = [None] * 10
win[0] = 2
for i in range(1, len(win)):
    win[i] = 2 # состояние по умолчанию проигрышное
    for k in [1, 2, 3]:
        if i - k >= 0 and win[i - k] == 2:
            win[i] = 1 # но если есть переход в проигрышное, то оно выигрышное
print(win)

[2, 1, 1, 1, 2, 1, 1, 1, 2, 1]


### Задание
Решите 3 первые задачи в этом контесте:

https://informatics.msk.ru/mod/statements/view3.php?id=33386


## Игра как граф
В общем случае состояния игры могут представлять из себя гораздо более сложную структуру, чем "N спичек". А именно, представим себе ориентированный граф:
* Вершины - это состояния игры
* Ребра - это переходы между ними
* В одной из вершин стоит фишка - текущее состояние игры
* Два игрока по очереди двигают фишку по одному из ребер.
* Кто не может двигать фишку - тот проиграл

На таком графе так же легко считать проигрышные и выигрышные состояния. А именно, вершины, из которых не выходят ребра - очевидно проигрышные. Вершины с ребрами в проигрышные - это выигрышные. А вершины с ребрами только в выигрышные - это проигрышные. 

Очень удобно сделать **топологическую сортировку** графа. После этого все ребра будут смотреть в одну сторон, и можно за один проход по массиву посчитать выигрышность и проигрышность каждой вершины. 

Но! Может ничего не выйти, если в графе есть цикл. В циклическом графе могут быть ничейные вершины - из которых у обоих игроков есть стратегия, при которой игра будет длиться вечно. 

Поэтому обычно рассматривают **конечные ациклические игры**, в которых граф конечен и ацикличен. Благодаря тому, что для них работает топологическая сортировка и переход динамики для подсчета выигрышных и проигрышных состояний, мы уже практически доказали, что в них **в каждой вершине существует либо выигрышная стратегия у первого игрока, либо у второго игрока**.

### Задание
Посчитайте у каждого состояние в этом графе-игре, выигрышное оно или нет. А также найдите для каждой вершины, куда оттуда идти оптимальнее всего:

<img src="https://drive.google.com/uc?export=view&id=17AoKkTRwzaj5s85_R9rvbuXjcmu0l85T">

### Решение

<img src="https://drive.google.com/uc?export=view&id=1k2NrpN17zJK7twYIGVL_RwCPRuOx5SXs">

Расмотрим такие игры:


1. Есть шахматная доска NxM, в верхней правой клетке стоит  ферзь. Два игрока по очереди двигают его либо вниз, либо влево, либо вниз-влево по диагонали на сколько угодно клеток. Проигрывает тот, кто не может ходить.
2. Крестики-нолики на доске 3x3.
3. Шахматы.


### Задание
Придумайте, как представить эти игры в виде графов. Что является состоянием игры? Являются ли игры ациклическими?

### Решение

1. Состояние игры - это пара (x, y) - координаты ферзя. Есть ходы из (x, y) в (x - k, y), (x, y - k) и (x - k, y - k), где k  - любое. Игра ациклическая - после каждого хода обе координаты не возрастают, и хотя бы одна убывает.
2.  Состояние игры - это доска с уже поставленными крестиками и ноликами. По доске однозначно считается, кто ходит следующий. Во все возможные доски, в которые можно походить, в графе стоит ребро. Игра ациклическая - после каждого хода заполняется езе одна клетка.
3. Состояние игры - это пара (доска с фигурами, чей сейчас ход). Особенность шахмат в том, что здесь бывают ничьи. Можно расширить нашу модель графа, введя ничейные вершины. А можно поставить во всех ничейных вершинах петлю - если ты попал в нее, то ты будешь вечно ходить по петле, и в соответствии с нашим старым определением, это действительно ничья. 

  Еще шахматы циклические - игроки могут вернуться в то же положение. Ха, вообще-то нет! Есть правило, согласно которому одно положение не может повторяться более 3 раз. Поэтому можно сделать состоянием игры (доска с фигурами, чей сейчас ход, статистика о том сколько раз встретилось каждая доска). При этом, если какая-то доска встретилась 3 раза, объявляем ничью. Тогда игра становится ациклической - статистика всегда в одной доске возрастает. При всем огромном числе возможных досок шахматы - игра конечная. Досок конечное число, и на каждой можно побывать не более 6 раз (3 раза с ходом белым, 3 раза с ходом черных), это все еще конечное число. А значит, в шахматах точно существует либо выигрышная стратегия для белых, либо для черных, либо ничейная у обоих.
  
### Задание
Решите 4, 5 и 6 задачи в этом контесте:

https://informatics.msk.ru/mod/statements/view3.php?id=33386

В них самое сложное - придумать, чем является **состояние игры**. Также рекомендуем использовать ленивую динамику, чтобы не думать о том, в каком порядке проходить состояния.

## Результат игры
  В прошлом разделе мы столкнулись с тем, что помимо проигрышных и выгрышных состояний бывают ничейные. Очень часто в играх вместо таких однозначных итогов вводят гораздо более сложные и интересные итоги. Например, можно считать, что результат игры - это пара чисел (A, B), где A - это выигрыш (в условных монетах) первого игрока, а B - выигрыш второго игрока. И оба игрока стремятся максимизировать свой выигрыш.
  
  Очень часто рассматриваются игры с нулевой суммой: A + B = 0, то есть сколько монет выиграл один, столько проиграл второй
  
  Можно считать, что в каждой вершине-листе (то есть откуда некуда идти) написан результат игры: первое число получает тот, кто должен ходить из листа, а второе число должен писать тот, кто туда пришел.
  
  Для такой модели существует аналог выигрышных-проигрышных позиций. А именно, давайте в win[x] хранить пару - лучший результат первого игрока, если он начнет играть с этой вершины, и сколько в этом случае получит второй игрок. Если это игра с нулевой суммой, то второй игрок потеряет ровно столько, сколько получит первый, в этом случае можно не хранить пару, а только выигрыш первого.
  
  Тогда такая динамика пересчитывается просто - если ты хочешь узнать результат игры в вершине X, то нужно просто максимизировать выигрыш **второго** игрока во всех состояниях, куда можно прийти из X. При этом результат **первого** игрока становится результатом второго игрока в вершине X. Конечно, при этом писать топологическую сортировку не надо, достаточно считать динамику прямо ленивой динамикой (DFS по графу игры).


### Задание
Посчитайте у каждого состояние в этом графе-игре результат игры. А также найдите для каждой вершины, куда оттуда идти оптимальнее всего:

<img src="https://drive.google.com/uc?export=view&id=1Mkkq9qgB0ygyj4YqXSae6TWlCvh2pZ9Q">

### Решение

<img src="https://drive.google.com/uc?export=view&id=1D7DmZs7b7dSTxxnbN3lsZoQz-KAdOVjT">


В общем, может быть сколько угодно моделей игры. Монетки могут получаться не только в листьях - они могут лежать на ребрах, и игрок получает их, если проходит по ребру, или в вершинах. Но идеи подсчета результата игры можно использовать и в таких задачах: **главное - понимать, что такое состояние в динамике.**

### Задание
Решите 2 последние задачи в этом контесте:

https://informatics.msk.ru/mod/statements/view3.php?id=33386